In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "slts-8e29d-firebase-adminsdk.json"


In [2]:
from google.cloud import firestore

db = firestore.Client()

users_ref = db.collection("userData")
users = users_ref.stream()

updated = 0
skipped = 0

for user_doc in users:
    user_data = user_doc.to_dict()

    # Only judges
    if user_data.get("role") != "judge":
        skipped += 1
        continue

    event_ref = user_data.get("eventRef")
    if not event_ref:
        print(f"⚠️ Skipping {user_doc.id}: no eventRef")
        skipped += 1
        continue

    # ✅ FIX: eventRef is a DocumentReference
    event_code = event_ref.id

    event_doc = db.collection("eventJudgeMapping").document(event_code).get()
    if not event_doc.exists:
        print(f"⚠️ Skipping {user_doc.id}: eventJudgeMapping missing for {event_code}")
        skipped += 1
        continue

    judge_order = event_doc.to_dict().get("judgeOrder", [])
    if not judge_order:
        print(f"⚠️ Skipping {user_doc.id}: no judges found")
        skipped += 1
        continue

    judge_name = judge_order[0]["name"]  # first judge

    # Update ONLY name
    users_ref.document(user_doc.id).update({
        "name": judge_name
    })

    print(f"✅ Updated {user_doc.id} → {judge_name}")
    updated += 1

print("\n🎯 DONE")
print("Updated:", updated)
print("Skipped:", skipped)


✅ Updated 03Eb5bx0vDVcpjSV1mtvwSXFYr93 → Rajagopal (TNN Spiritual Coordinator)
✅ Updated 0Lr3A2hMdlU1gXzva9KEYVrXJjx1 → Sai Narayanan
✅ Updated 0s8p7MRQdVUwyM3Ooy4GNtdeNQh2 → Sumitha Ganeshan (ATP Alumna)
⚠️ Skipping 18Tsl3misUcb3PQz09P92kBdsen2: eventJudgeMapping missing for GROUP-Altar-Decoration-Girls
⚠️ Skipping 1SVtJrpwz3gBiRwnzVkwwbYYpDr1: eventJudgeMapping missing for GROUP-Devotional-Singing-Boys
✅ Updated 1b59rUxYulRKcBBM5092b3MqzSz1 → Niranjan V.
✅ Updated 20aa5q24ZiX5Hox0Gvig4YmEl8s2 → Srividhya Ramesh (ATP Alumna + BV Guru)
⚠️ Skipping 2CkBzUrAlYTjaCnqSZdkNVD98O82: eventJudgeMapping missing for GROUP-Devotional-Singing-Girls
✅ Updated 2H68GW5biGhIcX9wX1cXh0EqsvX2 → Sai Prasath
⚠️ Skipping 2HKQMI3z8NSoojQ1RoDactuJmfq1: eventJudgeMapping missing for GROUP-Altar-Decoration
⚠️ Skipping 2mncTdnDkLclTImmp2uMtNtvxMq2: eventJudgeMapping missing for G2-Elocution-Tamil
✅ Updated 3kPDiVKl1aauQpp4A96p75UryUY2 → Ganapathi (TNS Spiritual Coordinator)
✅ Updated 4q9oSAlsXgOHt2Vd44INYnoZqio